In this notebook I will test the rouge metric for evaluating poetry

file:///Users/jbt694/Downloads/multilingual-extension-and-evaluation-of-a-poetry-generator-.pdf

https://kavita-ganesan.com/rouge-howto/

https://en.wikipedia.org/wiki/ROUGE_(metric)

In [1]:
from rouge_metric import PyRouge

import pandas as pd
import numpy as np

In [2]:
hypotheses = ["A big green desk"]
#hypotheses = ["I was having a beer"]


references = ["A big red desk"]

# Evaluate document-wise ROUGE scores
rouge = PyRouge(rouge_n=(1, 2, 3, 4),
                rouge_l=True,
                rouge_w=False,
                #rouge_w_weight=1.2,
                rouge_s=False,
                rouge_su=True,
                 skip_gap=4
               )
scores = rouge.evaluate(hypotheses, [references])
scores

{'rouge-1': {'r': 0.75, 'p': 0.75, 'f': 0.75},
 'rouge-2': {'r': 0.3333333333333333,
  'p': 0.3333333333333333,
  'f': 0.3333333333333333},
 'rouge-3': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.75, 'p': 0.75, 'f': 0.75},
 'rouge-su4': {'r': 0.5555555555555556,
  'p': 0.5555555555555556,
  'f': 0.5555555555555556}}

In [3]:
rouge = PyRouge(rouge_n=(1, 2, 3, 4),
                rouge_l=True,
                rouge_w=False,
                rouge_s=False,
                rouge_su=True,
                 skip_gap=4
               )


### Compare poem1 with poem2 line by line

In [4]:
f=open("data/poem1.txt")
lines1 = [line.strip() for line in f if len(line.strip())>0]
f.close()

f=open("data/poem2.txt")
lines2 = [line.strip() for line in f if len(line.strip())>0]
f.close()


results = []
for indline, (line1,line2) in enumerate(zip(lines1,lines2)):
    scores = rouge.evaluate([line1], [[line2]])
    results.append(scores)
    

FileNotFoundError: [Errno 2] No such file or directory: 'data/poem1.txt'

In [ ]:
accumulated_results=pd.DataFrame(results[0])
for results_line in results[1:]:
    accumulated_results += pd.DataFrame(results_line)

In [ ]:
accumulated_results

#This is so impressive. There is no overlap at all between these two poems.

Let's now make a test evaluating a poem that has been created based on another one.

In [ ]:
#poem2-slightlyplagiarized.txt

f=open("data/poem2.txt")
lines1 = [line.strip() for line in f if len(line.strip())>0]
f.close()

f=open("data/poem2-slightlyplagiarized.txt")
lines2 = [line.strip() for line in f if len(line.strip())>0]
f.close()

results = []
for indline, (line1,line2) in enumerate(zip(lines1,lines2)):
    scores = rouge.evaluate([line1], [[line2]])
    results.append(scores)

accumulated_results=pd.DataFrame(results[0])
for results_line in results[1:]:
    accumulated_results += pd.DataFrame(results_line)
    
accumulated_results

We have to normalize the results with the number of lines

In [ ]:
accumulated_results/len(results)

Yes... we can see that there is a bit of plagiarization.

### Compare poem1 and poem2
#### line1 with line1,2,3,4...
#### line2 with line1,2,3,4...

In [ ]:
#poem2-slightlyplagiarized.txt

f=open("data/poem2.txt")
lines1 = [line.strip() for line in f if len(line.strip())>0]
f.close()

f=open("data/poem2-slightlyplagiarized.txt")
lines2 = [line.strip() for line in f if len(line.strip())>0]
f.close()

results = []
for indline1, line1 in enumerate(lines1):
    for indline2,line2 in enumerate(lines2):
        scores = rouge.evaluate([line1], [[line2]])
        results.append(scores)

accumulated_results=pd.DataFrame(results[0])
for results_line in results[1:]:
    accumulated_results += pd.DataFrame(results_line)
    
accumulated_results/len(results)

### Compare poem1 and poem2 without considering the lines

#### I will concatenate all the lines together with " "

In [ ]:
#poem2-slightlyplagiarized.txt

f=open("data/poem2.txt")
poem1 = " ".join([line.strip() for line in f if len(line.strip())>0])
f.close()

f=open("data/poem2-slightlyplagiarized.txt")
poem2 = " ".join([line.strip() for line in f if len(line.strip())>0])
f.close()

scores = rouge.evaluate([poem1], [[poem2]])

pd.DataFrame(scores)


### Check one poem for its intra novelty

Now let's check the intra novelty.

In [ ]:
f=open("data/poem2.txt")
lines = [line.strip() for line in f if len(line.strip())>0]
f.close()

results = []
for indline1, line1 in enumerate(lines):
    for indline2, line2 in enumerate(lines):
        if indline1 != indline2:
            scores = rouge.evaluate([line1], [[line2]])
            results.append(((indline1,indline2),scores))

In [ ]:
results

In [ ]:
accumulated_results=pd.DataFrame(results[0][1])
for results_line in results[1:]:
    accumulated_results += pd.DataFrame(results_line[1])
    
accumulated_results

In [ ]:
accumulated_results/len(results)

In [ ]:
!cat ../data/poem2.txt

In [9]:
!ls ../data/eus_poems

bertsoa1.txt bertsoa2.txt bertsoa3.txt bertsoa4.txt bertsoa5.txt bertsoa6.txt


In [11]:
import glob

In [13]:
final_result=[]
poemlist=list(glob.glob("../data/eus_poems/*.txt"))
for filename1 in poemlist:
    for filename2 in poemlist:
        if filename1!=filename2:
            f=open(filename1)
            lines1 = [line.strip() for line in f if len(line.strip())>0]
            f.close()

            f=open(filename2)
            lines2 = [line.strip() for line in f if len(line.strip())>0]
            f.close()


            results = []
            for indline, (line1,line2) in enumerate(zip(lines1,lines2)):
                scores = rouge.evaluate([line1], [[line2]])
                results.append(scores)

            accumulated_results=pd.DataFrame(results[0])
            for results_line in results[1:]:
                accumulated_results += pd.DataFrame(results_line)

            final_result.append(accumulated_results/len(lines1))
            
            
accumulated_final_result=pd.DataFrame(final_result[0])
for results_line in final_result[1:]:
    accumulated_final_result += pd.DataFrame(results_line)
accumulated_final_result/((len(poemlist)-1) * (len(poemlist)))

,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rouge-su4
r,0.010111,0.002222,0.0,0.0,0.010111,0.002963
p,0.010111,0.002222,0.0,0.0,0.010111,0.002963
f,0.010000,0.002222,0.0,0.0,0.010000,0.002963
